In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI as genai
from langchain_community.document_loaders import WebBaseLoader
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings as GeminiEmbeddings

In [ ]:
load_dotenv()
_DEFAULT_EMBEDDINGS_MODEL= "models/embedding-001"
# Initializes a large language model (LLM) for text generation
llm = genai(model="models/gemini-2.0-flash-exp",
            temperature=0.6,
             top_p=0.85,
            )

In [ ]:
question = "what are the most recent udpates about hurricane Helene as of September 2024?"
response = llm.invoke(question)
print(response.content)

In [ ]:
url = "https://www.cnn.com/weather/live-news/hurricane-helene-florida-north-carolina-georgia-09-30-24/index.html"

loader = WebBaseLoader(url)

docs = loader.load()

In [ ]:
print(docs[0].page_content)

In [ ]:
embeddings = GeminiEmbeddings(model=_DEFAULT_EMBEDDINGS_MODEL)
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

** Context **
{context}

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [25]:
response = retrieval_chain.invoke({"input": question})

In [26]:
print(f"Prompt/> {question}\n\n", f"Answer: {response['answer']}")

Prompt/> what are the most recent udpates about hurricane Helene as of September 2024?

 Answer: Here are the most recent updates about Hurricane Helene as of September 2024, based on the provided text:

*   **Buncombe County confirms 35 storm deaths:** Buncombe County, North Carolina, now has 35 confirmed deaths from Hurricane Helene.
*   **Helene death toll reaches 115:** There are now 115 confirmed deaths from Helene in six states.
*   **Florida urban search teams to end operations:** Urban search and rescue teams will finish all their operations in the state of Florida by the end of the day Monday. Florida is focusing on deploying recovery assistance to hard-hit areas in North Carolina and Tennessee.
*   **President Biden calls Helene "history-making":** President Biden called Helene “a history-making storm” and said he hopes to visit impacted areas later this week. He vowed the federal government will assist survivors of Hurricane Helene “for as long as it takes."
*   **New storm 